In [37]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import itertools
from time import time
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from subprocess import Popen,PIPE,STDOUT
from sklearn.preprocessing import LabelEncoder

# Поменяйте на свой путь к данным
PATH_TO_DATA = '/tmp/working/machinlearning/6/ident/data/capstone_user'
KAGGLE_DATA = 'kaggle_data/'

with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'rb') as X_train_sparse_pkl:
    X_train_sparse = pickle.load(X_train_sparse_pkl)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'rb') as X_test_sparse_pkl:
    X_test_sparse = pickle.load(X_test_sparse_pkl)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'rb') as train_target_pkl:
    y = pickle.load(train_target_pkl)

class_encoder = LabelEncoder().fit(y)

y_for_vw = class_encoder.transform(y) + 1

X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y_for_vw, test_size=0.3, 
                                                     random_state=17, stratify=y_for_vw)

# Поменяйте на свой путь к данным
PATH_TO_DATA = '/tmp/working/machinlearning/6/ident/data/capstone_user/' 

train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model_full.vw')
model_part = os.path.join(PATH_TO_DATA, 'vw_model_part.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

train_vw_model(train_part_vw,model_part,num_classes=400)

test_vw_model(model_part,valid_vw,pred,y_valid,quiet=False)

def train_vw_model(train_vw_file, model_filename, num_classes=10,
                   bit_precision=28,l2=1e-8, passes=1,
                   seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw --oaa {num_classes} {train_vw_file} ' + 
                       '-f {model_filename} -b {bit_precision} --random_seed {seed}').format(
                       num_classes=num_classes, train_vw_file=train_vw_file, 
                       model_filename=model_filename, bit_precision=bit_precision, seed=seed)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    
    vw_call_string += " --l2 {}".format(l2)
    
    
    proc = Popen(vw_call_string,stdout=PIPE,shell=True)
    out,err=proc.communicate()
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not err else 'Failed.')
    return out

def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    proc = Popen(vw_call_string,stdout=PIPE,shell=True)
    out,err=proc.communicate()
        
    if not err: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("Accuracy: {}%. Elapsed: {} sec.".format(
            round(100 * accuracy_score(true_labels, vw_pred), 2), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')